# Test Best Models

Test notebook to generate the statistics of the different models found with AMS. First load the required libraries

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K
from keras.models import model_from_json

from sklearn.model_selection import KFold
from scipy import stats

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

Using TensorFlow backend.


### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler(data_scaler=None):

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 24
    window_stride = 1
    max_rul = 129

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features,
                                       max_rul, window_size, window_stride, data_scaler=data_scaler)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps

### Load models and evaluate model

In [4]:
def load_model(model_file, weights_file="", problem_type=1):
    
    p_type = ""
    
    # load json and create model
    json_file = open(model_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    print("Loaded model from disk") 
        
    #Load weights if specified
    if weights_file != "":
        # load weights into new model
        loaded_model.load_weights(weights_file)
        print("Loaded weights from disk") 
    else:
        print("Model needs training")
        
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        p_type = "regression"
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        p_type = "classification"
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    print("Created model for " + p_type + " with loss function " + lossFunction)

    return loaded_model
        
        

### Load each of the models and test them

In [5]:
best_model_folder = 'best_models'
"""
test_sets = {'cifar10':(CIFAR10DataHandler, None, 2), 
             'cmapss':(cmaps_dhandler, MinMaxScaler(feature_range=(-1, 1)), 1), 
             'mnist':((MNISTDataHandler), None, 2)}
"""

test_sets = {'mnist':((MNISTDataHandler), None, 2)}
#alpha_folders = ['alpha0.6', 'alpha0.8', 'alpha1']
#alpha_folders = ['alpha0.5']
alpha_values = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

In [6]:
results = {}

k = 5  #For 10-fold cross validation

for dataset in test_sets.keys():
    for alphaValue in alpha_values:
        
        model_file_name = 'scalarized/alpha{}/bestModel_global.json'.format(alphaValue)
        weights_file_name = 'scalarized/alpha{}/bestModel_global.h5'.format(alphaValue)
        trainable_count = 0
        
        print("Testing for "+model_file_name)
        
        results_key = dataset + "_" +  str(alphaValue)
        
        evaluations_cv = np.zeros(k)
        evaluations_test = np.zeros(k)
        
        model_location = best_model_folder + '/' + dataset + '/' + model_file_name
        
        if weights_file_name != "":
            weights_location = best_model_folder + '/' + dataset + '/' + weights_file_name
        
        dhandler, data_scaler, problem_type = test_sets[dataset]

        #model = load_model(model_location, weights_location, problem_type)   
        
        data_handler = dhandler(data_scaler=data_scaler)
        data_handler.load_data(verbose = 1)
        data_handler.print_data()
        
        folds = list(KFold(n_splits=k, shuffle=True).split(data_handler.X_train))
        
        print('Validation on model:' + model_location)
        
        for j, (train_idx, val_idx) in enumerate(folds):

            print('\nExperiment on Fold ', j)
            
            K.clear_session()  #Clear the previous tensorflow graph 

            X_train_cv = data_handler.X_train[train_idx]
            y_train_cv = data_handler.y_train[train_idx]
            X_valid_cv = data_handler.X_train[val_idx]
            y_valid_cv = data_handler.y_train[val_idx]

            model = load_model(model_location, "", problem_type)
            model.summary()
            trainable_count = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))

            model.fit(X_train_cv, y_train_cv, batch_size=512, epochs=100, verbose=1)

            evaluation_cv = model.evaluate(X_valid_cv, y_valid_cv)
            evaluation_test = model.evaluate(data_handler.X_test, data_handler.y_test)

            evaluations_cv[j] = evaluation_cv[1]
            evaluations_test[j] = evaluation_test[1]
            
        results[results_key] = (evaluations_cv, evaluations_test, trainable_count)
        

Testing for scalarized/alpha0.2/bestModel_global.json
Loading data. Cross-Validation ratio 0
Printing shapes

Training data (X, y)
(60000, 784)
(60000, 10)
Testing data (X, y)
(10000, 784)
(10000, 10)
Printing first 5 elements

Training data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Testing data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Validation on model:best_models/mnist/scalarized/alpha0.2/bestModel_global.json

Experiment on Fold  0
Loaded model from disk
Model needs training


48000/48000 [==============================] - 0s 6us/step - loss: 1.9509e-04 - acc: 1.0000
Epoch 64/100
48000/48000 [==============================] - 0s 6us/step - loss: 1.8633e-04 - acc: 1.0000
Epoch 65/100
48000/48000 [==============================] - 0s 6us/step - loss: 1.7530e-04 - acc: 1.0000
Epoch 66/100
48000/48000 [==============================] - 0s 6us/step - loss: 1.6230e-04 - acc: 1.0000
Epoch 67/100
48000/48000 [==============================] - 0s 6us/step - loss: 1.5424e-04 - acc: 1.0000
Epoch 68/100
48000/48000 [==============================] - 0s 6us/step - loss: 1.4562e-04 - acc: 1.0000
Epoch 69/100
48000/48000 [==============================] - 0s 6us/step - loss: 1.3652e-04 - acc: 1.0000
Epoch 70/100
48000/48000 [==============================] - 0s 6us/step - loss: 1.2898e-04 - acc: 1.0000
Epoch 71/100
48000/48000 [==============================] - 0s 6us/step - loss: 1.2153e-04 - acc: 1.0000
Epoch 72/100
48000/48000 [==============================] - 0s 6us/s

Epoch 33/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0022 - acc: 0.9999
Epoch 34/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0016 - acc: 1.0000
Epoch 35/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0013 - acc: 1.0000
Epoch 36/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0011 - acc: 1.0000
Epoch 37/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0011 - acc: 1.0000
Epoch 38/100
48000/48000 [==============================] - 0s 6us/step - loss: 9.9951e-04 - acc: 1.0000
Epoch 39/100
48000/48000 [==============================] - 0s 6us/step - loss: 9.3816e-04 - acc: 1.0000
Epoch 40/100
48000/48000 [==============================] - 0s 6us/step - loss: 8.7114e-04 - acc: 1.0000
Epoch 41/100
48000/48000 [==============================] - 0s 6us/step - loss: 8.2649e-04 - acc: 1.0000
Epoch 42/100
48000/48000 [==============================] - 0s 6us/step - l

48000/48000 [==============================] - 0s 8us/step - loss: 0.5790 - acc: 0.8451
Epoch 2/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.2223 - acc: 0.9358
Epoch 3/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.1689 - acc: 0.9506
Epoch 4/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.1367 - acc: 0.9607
Epoch 5/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.1143 - acc: 0.9670
Epoch 6/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0982 - acc: 0.9709
Epoch 7/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0837 - acc: 0.9756
Epoch 8/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0729 - acc: 0.9783
Epoch 9/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0646 - acc: 0.9807
Epoch 10/100
48000/48000 [==============================] - 0s 6us/step - loss: 0.0554 - acc: 0.9832
Epoch 11/10

48000/48000 [==============================] - 0s 6us/step - loss: 6.1563e-05 - acc: 1.0000
Epoch 81/100
48000/48000 [==============================] - 0s 6us/step - loss: 5.7956e-05 - acc: 1.0000
Epoch 82/100
48000/48000 [==============================] - 0s 6us/step - loss: 5.5462e-05 - acc: 1.0000
Epoch 83/100
48000/48000 [==============================] - 0s 6us/step - loss: 5.1971e-05 - acc: 1.0000
Epoch 84/100
48000/48000 [==============================] - 0s 6us/step - loss: 4.8994e-05 - acc: 1.0000
Epoch 85/100
48000/48000 [==============================] - 0s 6us/step - loss: 4.6195e-05 - acc: 1.0000
Epoch 86/100
48000/48000 [==============================] - 0s 6us/step - loss: 4.4560e-05 - acc: 1.0000
Epoch 87/100
48000/48000 [==============================] - 0s 6us/step - loss: 4.1500e-05 - acc: 1.0000
Epoch 88/100
48000/48000 [==============================] - 0s 6us/step - loss: 3.9481e-05 - acc: 1.0000
Epoch 89/100
48000/48000 [==============================] - 0s 6us/s

...

48000/48000 [==============================] - 0s 3us/step - loss: 0.2108 - acc: 0.9402
Epoch 99/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2102 - acc: 0.9406
Epoch 100/100
10000/10000 [==============================] - 0s 5us/step

Experiment on Fold  2
Loaded model from disk
Model needs training
Created model for classification with loss function categorical_crossentropy
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 8)                 6280      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                90        
Total params: 6,370
Trainable params: 6,370
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
48000/48000 [==============================] - 0s 5us/step - loss: 1.5501 - acc: 0.5383
Epoch 2/100
48000/48000 [=====

48000/48000 [==============================] - 0s 3us/step - loss: 0.2370 - acc: 0.9329
Epoch 71/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2364 - acc: 0.9327
Epoch 72/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2356 - acc: 0.9330
Epoch 73/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2357 - acc: 0.9336
Epoch 74/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2349 - acc: 0.9330
Epoch 75/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2346 - acc: 0.9334
Epoch 76/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2339 - acc: 0.9336
Epoch 77/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2333 - acc: 0.9334
Epoch 78/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2328 - acc: 0.9339
Epoch 79/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2327 - acc: 0.9338
Epo

48000/48000 [==============================] - 0s 3us/step - loss: 0.2426 - acc: 0.9310
Epoch 43/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2418 - acc: 0.9316
Epoch 44/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2403 - acc: 0.9318
Epoch 45/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2399 - acc: 0.9320
Epoch 46/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2390 - acc: 0.9321
Epoch 47/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2382 - acc: 0.9323
Epoch 48/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2372 - acc: 0.9323
Epoch 49/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2362 - acc: 0.9329
Epoch 50/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2352 - acc: 0.9333
Epoch 51/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2349 - acc: 0.9323
Epo

48000/48000 [==============================] - 0s 3us/step - loss: 0.2936 - acc: 0.9171
Epoch 15/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2892 - acc: 0.9181
Epoch 16/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2856 - acc: 0.9202
Epoch 17/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2823 - acc: 0.9210
Epoch 18/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2794 - acc: 0.9223
Epoch 19/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2768 - acc: 0.9227
Epoch 20/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2738 - acc: 0.9235
Epoch 21/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2715 - acc: 0.9242
Epoch 22/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2697 - acc: 0.9241
Epoch 23/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2672 - acc: 0.9257
Epo

48000/48000 [==============================] - 0s 3us/step - loss: 0.2069 - acc: 0.9415
Epoch 96/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2068 - acc: 0.9410
Epoch 97/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2065 - acc: 0.9420
Epoch 98/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2059 - acc: 0.9421
Epoch 99/100
48000/48000 [==============================] - 0s 3us/step - loss: 0.2054 - acc: 0.9420
Epoch 100/100
10000/10000 [==============================] - 0s 5us/step


### Print the stats

In [7]:
for key in results.keys():
    
    print("\nStats for: ")
    print(key)
    evaluations_cv, evaluations_test, model_size = results[key]
    
    print("\nModel size: %d"%model_size)
    
    print("\nCrossVal stats: ")
    print(evaluations_cv)
    print(stats.describe(evaluations_cv))
    
    print("\nTest stats: ")
    print(stats.describe(evaluations_test))


Stats for: 
mnist_0.2

Model size: 106330

CrossVal stats: 
[0.97683333 0.97591667 0.9765     0.97558333 0.97725   ]
DescribeResult(nobs=5, minmax=(0.9755833333333334, 0.97725), mean=0.9764166666666666, variance=4.548611111110849e-07, skewness=-0.04745430034285107, kurtosis=-1.403210768603311)

Test stats: 
DescribeResult(nobs=5, minmax=(0.9767, 0.9785), mean=0.9778400000000002, variance=5.580000000000081e-07, skewness=-0.6974994435346632, kurtosis=-0.9977357690671402)

Stats for: 
mnist_0.3

Model size: 161962

CrossVal stats: 
[0.979      0.98058333 0.98025    0.98358333 0.98008333]
DescribeResult(nobs=5, minmax=(0.979, 0.9835833333333334), mean=0.9806999999999999, variance=2.9500000000000785e-06, skewness=1.0332748778123741, kurtosis=-0.21150415083978036)

Test stats: 
DescribeResult(nobs=5, minmax=(0.9812, 0.9822), mean=0.9819000000000001, variance=1.799999999999992e-07, skewness=-1.0321322918616536, kurtosis=-0.5115740740724712)

Stats for: 
mnist_0.4

Model size: 25570

CrossVal